In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# test

In [3]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torchvision import datasets,transforms

In [4]:
import torch
import numpy as np
import skimage
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(1)  # reproducible
torch.set_default_tensor_type(torch.DoubleTensor)
from PIL import Image
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
])
'''NPY数据格式'''
# class MyDataset(Dataset):
#     def __init__(self, data,label):
#         self.data = np.load(data) #加载npy数据
#         # self.data=self.data
#         self.label = np.load(label)
#         self.transforms = transform #转为tensor形式
#     def __getitem__(self, index):
#         hdct= self.data[index, :, :, :]  # 读取每一个npy的数据
#         # print(hdct.shape)
#         # hdct = np.squeeze(hdct)  # 删掉一维的数据，就是把通道数这个维度删除
#         # ldct = 2.5 * skimage.util.random_noise(hdct * (0.4 / 255), mode='poisson', seed=None) * 255 #加poisson噪声
#         # hdct=Image.fromarray(np.uint8(hdct)) #转成image的形式
#         # ldct=Image.fromarray(np.uint8(ldct)) #转成image的形式
#         hdct = hdct.astype("float")
#         hdct= self.transforms(hdct)  #转为tensor形式
#         # ldct= self.transforms(ldct)  #转为tensor形式？??
#         return hdct, self.label[index] #返回数据还有标签
#     def __len__(self):
#         return self.data.shape[0] #返回数据的总个数
 
# def main():
#     dataset=MyDataset('train1.npy',"label.npy")
#     data= DataLoader(dataset, batch_size=8, shuffle=True, pin_memory=True)
# if __name__ == '__main__':
#     main()

'NPY数据格式'

In [5]:

class MyDataset(Dataset):
    def __init__(self, data,label):
        self.data = data #加载npy数据
        self.label = label
        self.transforms = transform #转为tensor形式
    def __getitem__(self, index):
        hdct= self.data[index, :, :, :]  # 读取每一个npy的数据
        hdct = np.squeeze(hdct)  # 删掉一维的数据，就是把通道数这个维度删除
#         ldct = 2.5 * skimage.util.random_noise(hdct * (0.4 / 255), mode='poisson', seed=None) * 255 #加poisson噪声
#         hdct=Image.fromarray(np.uint8(hdct)) #转成image的形式
#         ldct=Image.fromarray(np.uint8(ldct)) #转成image的形式
        hdct= self.transforms(hdct)  #转为tensor形式
#         ldct= self.transforms(ldct)  #转为tensor形式
        return hdct, self.label[index] #返回数据还有标签
    def __len__(self):
        return self.data.shape[0] #返回数据的总个数



In [6]:
# dataset=MyDataset('/content/drive/MyDrive/100/test3/x_train.npy',"/content/drive/MyDrive/100/test3/y_train.npy")
# dataset_test=MyDataset('/content/drive/MyDrive/100/test3/x_test.npy',"/content/drive/MyDrive/100/test3/y_test.npy")
# train_loader= DataLoader(dataset, batch_size=128, shuffle=True, pin_memory=True)
# test_loader= DataLoader(dataset_test, batch_size=128, shuffle=False, pin_memory=True)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5,padding=2)
        self.conv2 = nn.Conv2d(6, 12, kernel_size=5)
        self.mp = nn.MaxPool2d(3)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(6912,640)
        self.fc2 = nn.Linear(640,64)
        self.fc3 = nn.Linear(64,8)
        self.logsoftmax = nn.LogSoftmax(dim=1)
        
    def forward(self,x):
        in_size = x.size(0)
        out = self.relu(self.mp(self.conv1(x)))
        out = self.relu(self.mp(self.conv2(out)))
        out = out.view(in_size, -1)
        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        return self.logsoftmax(out)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
device

device(type='cuda', index=0)

In [10]:
model = Net().to(device)

In [11]:
loss_fn = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.008,momentum=0.5)

In [12]:
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold as kFold

In [13]:
x_train=np.load("/content/drive/MyDrive/summer_start/data_generate/train_3channel.npy")
y_train=np.load("/content/drive/MyDrive/summer_start/data_generate/label.npy")

In [14]:
# da1=np.load('/content/drive/MyDrive/100/test3/x_test.npy')
x_train=x_train.reshape(13887,236,236,-1)

In [15]:
x_train.shape

(13887, 236, 236, 3)

In [16]:
kfold =kFold(n_splits=5,shuffle=True,random_state=1)

In [17]:
# for epoch in range(40):

#   list1=[]
#   for t, (data, target) in enumerate(train_loader):
#     data,target = Variable(data.to(device)),Variable(target.to(device))
#     pred = model(data).to(device)
#     loss = loss_fn(pred,target)
#     list1.append(loss.item())
    
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     # print(t,loss.item())
#   print(epoch,np.mean(list1))
#   if epoch%2==0:
#     correct = 0
#     for data, target in test_loader:
#         data, target = Variable(data.to(device), volatile=True), Variable(target.to(device))
#         output = model(data.to(device))
#         # get the index of the max log-probability
#         pred = output.data.max(1, keepdim=True)[1]
#         correct += pred.eq(target.data.view_as(pred)).cpu().sum()
#     print('{:.3f}%\n'.format(
#         100. * correct / len(test_loader.dataset)))
#     correct=correct / len(test_loader.dataset)
    

In [18]:
import time

In [19]:
labels=[0,1,2,3,4,5,6,7]
list_score=[]
for fold, (train_index, test_index) in enumerate(kfold.split(x_train, y_train)):
    print("train_index",train_index)
    print("test_index",test_index)
    ### Dividing data into folds
    x_train_fold = x_train[train_index]
    x_test_fold = x_train[test_index]
    y_train_fold = y_train[train_index]
    y_test_fold = y_train[test_index]
    train = MyDataset(x_train_fold, y_train_fold)
    test = MyDataset(x_test_fold, y_test_fold)
    train_loader = DataLoader(train, batch_size = 16, shuffle = True)
    test_loader = DataLoader(test, batch_size = 16, shuffle = True)
    print("fold:",fold)
    max_score=[]
    # model = Net().to(device)
    best_acc=0
    model = Net().to(device)
    loss_fn = nn.NLLLoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.5)
    for i in range(50):
      list1=[]
      for t, (data, target) in enumerate(train_loader):
        # print("t",t)
        data,target = Variable(data.to(device)),Variable(target.to(device))
        # print(data.shape)
        pred = model(data.double().to(device))
        loss = loss_fn(pred,target)
        
        list1.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print("loss",loss.item())
      print("netloss",i,np.mean(list1))
      with torch.no_grad():
        list_loss=[]
        correct = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data.double().to(device))
            # get the index of the max log-probability
            pred = output.data.max(1, keepdim=True)[1]
            pred_loss=np.exp(output.data.cpu())
            logsloss=log_loss(target.cpu().detach().numpy(),pred_loss,labels=labels)
            list_loss.append(logsloss)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        print('accuracy:{:.3f}%'.format(
            100. * correct / len(test_loader.dataset)))
        val_acc=correct / len(test_loader.dataset)
        print("log_loss",sum(list_loss)/len(list_loss))
        print("*"*20)
        if val_acc > best_acc:
            best_acc = val_acc
            es = 0
        else:
            es += 1
            print("Counter {} of 5".format(es))
            if es > 5:
                print("Early stopping with best_acc: ", best_acc, "and val_acc for this epoch: ", val_acc, "...")
                break
    list_score.append(best_acc)
    print("*"*40)
    del train,x_train_fold,train_loader,test_loader,test,y_test_fold,x_test_fold,y_train_fold
    import gc
    gc.collect()
    time.sleep(20)
from numpy import mean
print(mean(list_score))


train_index [    0     1     2 ... 13884 13885 13886]
test_index [    4     5     6 ... 13861 13872 13877]
fold: 0
netloss 0 1.8691641769543919
accuracy:52.448%
log_loss 1.7435952758425086
********************
netloss 1 1.6600124340841864
accuracy:56.623%
log_loss 1.4868107853443975
********************
netloss 2 1.5428666919142746
accuracy:57.235%
log_loss 1.2779947152655566
********************
netloss 3 1.2840860557429308
accuracy:58.171%
log_loss 1.2541444226507064
********************
netloss 4 1.1854866594746585
accuracy:60.007%
log_loss 1.167604845245739
********************
netloss 5 1.1973601958604208
accuracy:62.275%
log_loss 1.1402553409169294
********************
netloss 6 1.06408982761749
accuracy:65.515%
log_loss 1.0499554113922005
********************
netloss 7 1.1230836636054218
accuracy:66.703%
log_loss 1.021400976396502
********************
netloss 8 0.9761432572735188
accuracy:65.875%
log_loss 1.0218907750881516
********************
Counter 1 of 5
netloss 9 0.9157563

In [22]:
mean(list_score)

0.8407849835102856